# Spotify Genre and Decade Predictor
This app predicts genre and decade for a given song/track and artist using Random Forest Classifier models.    
***

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from IPython.display import display, clear_output

from sklearn.ensemble import RandomForestClassifier
import joblib

#Load the Trained Model and the fit Scaler
rfModel_decade = joblib.load('rfModel.joblib')
scaler_decade = joblib.load('scaler.joblib')

rfModel_genre = joblib.load('rfModel_genre.joblib')
scaler_genre = joblib.load('scaler_genre.joblib')

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import ipywidgets as widgets
from IPython.display import display

In [3]:
# Set up your Spotify API credentials
client_id = '7f37bb631ed34cf29a487f88d5c5e32f'
client_secret = 'aff5f020a49b470f99d7a3c251f5b649'

# Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define a function to get track audio features
def get_track_audio_features(artist_name, track_name):
    # Search for the track
    results = sp.search(q=f"artist:{artist_name} track:{track_name}", type='track')

    # Check if the search results contain any tracks
    if len(results['tracks']['items']) == 0:
        print(f"No track found for '{artist_name} - {track_name}'")
        return None

    # Get the first track from the search results
    track = results['tracks']['items'][0]

    # Get the track's ID
    track_id = track['id']

    # Get audio features for the track
    audio_features = sp.audio_features(track_id)

    # Extract desired features
    track_features = {}
#    track_features['Artist'] = artist_name
#    track_features['Track'] = track_name
    track_features['Valence'] = audio_features[0]['valence']
    track_features['Acousticness'] = audio_features[0]['acousticness']
    track_features['Danceability'] = audio_features[0]['danceability']
    track_features['Duration (ms)'] = audio_features[0]['duration_ms']
    track_features['Energy'] = audio_features[0]['energy']
    track_features['Explicit'] = track['explicit']
    track_features['Instrumentalness'] = audio_features[0]['instrumentalness']
    track_features['Key'] = audio_features[0]['key']
    track_features['Liveness'] = audio_features[0]['liveness']
    track_features['Loudness'] = audio_features[0]['loudness']
    track_features['Mode'] = audio_features[0]['mode']
    track_features['Popularity'] = track['popularity']
    track_features['Speechiness'] = audio_features[0]['speechiness']
    track_features['Tempo'] = audio_features[0]['tempo']
    track_features['Time Signature'] = audio_features[0]['time_signature']
    
    # Get the release date of the track
    release_date = track['album']['release_date']
    actual_year = release_date.split('-')[0]       # Extract the year from the release date

    return track_features, actual_year


In [22]:
def predict_decade(track_features): 
    # Create a new dictionary to store a decade-model-specific dictionary
    track_features_decade = {key: value for key, value in track_features.items() if key != 'Time Signature'}
    
    # Convert the dictionary to an array
    feature_array_decade = [track_features_decade[key] for key in track_features_decade]
    
    feature_array_decade = np.array(feature_array_decade).reshape(1, -1)
    spotify_track_scaled_decade = scaler_decade.transform(feature_array_decade)  # Preprocess the sample data using the scaler

    # Make the prediction
    predicted_decade = rfModel_decade.predict(spotify_track_scaled_decade)
    
    return predicted_decade


In [23]:
def predict_genre(track_features):     
    # Rearrange track_features to match the genre rfModel
    track_features_genre = {}
    track_features_genre['Popularity'] = track_features['Popularity']
    track_features_genre['Danceability'] = track_features['Danceability']
    track_features_genre['Energy'] = track_features['Energy']
    track_features_genre['Key'] = track_features['Key']
    track_features_genre['Loudness'] = track_features['Loudness']
    track_features_genre['Mode'] = track_features['Mode']
    track_features_genre['Speechiness'] = track_features['Speechiness']
    track_features_genre['Acousticness'] = track_features['Acousticness']
    track_features_genre['Instrumentalness'] = track_features['Instrumentalness']
    track_features_genre['Liveness'] = track_features['Liveness']
    track_features_genre['Valence'] = track_features['Valence']
    track_features_genre['Tempo'] = track_features['Tempo']
    track_features_genre['Duration (ms)'] = track_features['Duration (ms)']
    track_features_genre['Time Signature'] = track_features['Time Signature']
    
    # Convert the dictionary to an array
    feature_array_genre = [track_features_genre[key] for key in track_features_genre]
    
    feature_array_genre = np.array(feature_array_genre).reshape(1, -1)
    spotify_track_scaled_genre = scaler_genre.transform(feature_array_genre)  # Preprocess the sample data using the scaler

    # Make the prediction
    prediction = rfModel_genre.predict(spotify_track_scaled_genre)

    # Convert prediction output to Genre
    genre_label = {
        0: 'Acoustic/Folk',
        1: 'Alt_Music',
        2: 'Blues',
        3: 'Bollywood',
        4: 'Country',
        5: 'HipHop',
        6: 'Indie Alt',
        7: 'Instrumental',
        8: 'Metal',
        9: 'Pop',
        10: 'Rock'
    }

    predicted_genre = genre_label[prediction[0]]
    
    return predicted_genre



In [ ]:
# Use HTML Markdown to style the headers
def generate_styled_header(header_text, paragraph_text):
    styled_header = '''
    <div style="background-color: lightgray; padding: 10px; font-family: Arial; text-align: center;">
        <h2 style="color: black;">{}</h2>
    </div>
    <div style="font-family: Arial; font-style: italic;">
        <p>{}</p>
    </div>
    '''.format(header_text, paragraph_text)
    return styled_header

def secondary_styled_header(header_text, paragraph_text):
    styled_header2 = '''
    <div style="background-color: white; padding: 10px; font-family: Arial; text-align: center;">
        <h2 style="color: light_gray; text-decoration: underline;">{}</h2>
    </div>
    <div style="font-family: Arial; font-style: italic;">
        <p>{}</p>
    </div>
    '''.format(header_text, paragraph_text)
    return styled_header2


In [30]:
# Create text input widgets
track_name_widget = widgets.Text(description='Track Name:')
artist_name_widget = widgets.Text(description='Artist Name:')

# Create button widget
button = widgets.Button(description='GET TRACK INFO')
# Modify the button style
button.style.button_color = 'lightblue'
button.layout.width = '200px'
button.layout.height = '50px'
button.layout.font_size = '16px'

output = widgets.Output()

# Define function to handle button click event
def on_button_click(b):
    track_name = track_name_widget.value
    artist_name = artist_name_widget.value       
    
    with output:
        clear_output(wait=True) 
        track_features, actual_year = get_track_audio_features(artist_name, track_name)
        predicted_decade = predict_decade(track_features)
        predicted_genre = predict_genre(track_features)

        print(f"Predicted decade for '{track_name}' by {artist_name}: {predicted_decade[0]}")
        print(f"Actual year for '{track_name}' by {artist_name}: {actual_year}")
        print("")
        print(f"Predicted genre for '{track_name}' by {artist_name}: {predicted_genre}")

    
# Attach button click event handler
button.on_click(on_button_click)

#info = Markdown("""# SPOTIFY DECADE PREDICTOR""")
makeDisplay = widgets.VBox([track_name_widget, artist_name_widget,  button, output])
display(makeDisplay)